# F. Explore Initial conditions restart or state file of SUMMA 3.0.0

## 1. Import pySUMMA 3.0.0

In [1]:
import pysumma as ps

## 2. Create Simulatioin Object

In [2]:
executable = "/usr/bin/summa.exe"
file_manager = 'SummaModel_ReynoldsAspenStand_StomatalResistance/settings/summa_fileManager_riparianAspenSimpleResistance.txt'

s = ps.Simulation(executable, file_manager)

## 3. Explore initial conditions, restart or state file

The initial conditions, restart, or state file is a NetCDF file that specifies the model states at the start of the model simulation. This file is required. You will need to generate one before you run the model for the first time, but after that the model restart file can be the result from an earlier model simulation. The file is written by **`build/source/netcdf/modelwrite.f90:writeRestart()`** and read by **`build/source/netcdf/read_icond.f90:read_icond_nlayers()`** (number of snow and soil layers) and **`build/source/netcdf/read_icond.f90:read_icond()`** (actual model states).

The frequency with which SUMMA writes restart files is specified on the command-line with the `-r` or `--restart` flag. This flag currently can be either `y` or `year`, `m` or `month`, `d` or `day`, or `n` or `never`.

As an input file, the variables that need to be specified in the restart file are a subset of those listed as `iLook_prog` in the `var_lookup` module in **`build/source/dshare/var_lookup.f90`** (look for the comment **`(6) define model prognostic (state) variables`**). Variable names must match the code exactly (case-sensitive). Note that not all the variables in `iLook_prog` need to be specified, since some of them can be calculated from other variables. For example, SUMMA calculates `mLayerHeight` from `iLayerHeight` and the variable does not need to be reported separately. For similar reasons, the user does not need to specify `scalarCanopyWat`, `spectralSnowAlbedoDiffuse`, `scalarSurfaceTemp`, `mLayerVolFracWat`, and `mLayerHeight` since these are skipped when the file is read and calculated internally to ensure consistency. In addition to these variables, the restart file also needs to specify the number of soil and snow layers (`nSoil` and `nSnow`, respectively).

The restart file does not have a time dimension, since it represents a specific moment in time. However, it has the following dimensions,: `hru, scalarv, spectral, ifcSoil, ifcToto, midSoil, and midToto`. These dimensions are described in detail in the section on `SUMMA output file dimensions` (keep in mind that the restart files are both input and output).

|Variable|	dimension|	type|	units|	long name|	
|:----------|:---------------|:----------|:----------|:-------------|
|dt_init|	scalarv, hru|	double|	seconds|	Length of initial time sub-step at start of next time interval (s)	|
|nSoil|	scalarv, hru|	int|	-|	Number of soil layers	|
|nSnow|	scalarv, hru|	int|	-|	Number of snow layers	|
|scalarCanopyIce|	scalarv, hru|	double|	kg m-2|	Mass of ice on the vegetation canopy	|
|scalarCanopyLiq|	scalarv, hru|	double|	kg m-2|	Mass of liquid water on the vegetation canopy	|
|scalarCanairTemp|	scalarv, hru|	double|	Pa|	Temperature of the canopy air space	|
|scalarCanopyTemp|	scalarv, hru|	double|	K|	Temperature of the vegetation canopy	|
|scalarSnowAlbedo|	scalarv, hru|	double|	-|	Snow albedo for the entire spectral band	|
|scalarSnowDepth|	scalarv, hru|	double|	m|	Total snow depth	|
|scalarSWE|	scalarv, hru|	double|	kg m-2|	Snow water equivalent	|
|scalarSfcMeltPond|	scalarv, hru|	double|	kg m-2|	Ponded water caused by melt of the "snow without a layer"	|
|scalarAquiferStorage|	scalarv, hru|	double|	m|	Relative aquifer storage -- above bottom of the soil profile	|
|iLayerHeight|	ifcToto, hru|	double|	m|	Height of the layer interface; top of soil = 0	|
|mLayerDepth|	midToto, hru|	double|	m|	Depth of each layer	|
|mLayerTemp|	midToto, hru|	double|	K|	Temperature of each layer	|
|mLayerVolFracIce|	midToto, hru|	double|	-|	Volumetric fraction of ice in each layer	|
|mLayerVolFracLiq|	midToto, hru|	double|	-|	Volumetric fraction of liquid water in each layer	|
|mLayerMatricHead|	midSoil, hru|	double|	m|	Matric head of water in the soil	|

### 3.1 Show initial conditioin file

In [3]:
initial_conditions = s.initial_conditions
initial_conditions

<xarray.Dataset>
Dimensions:               (hru: 1, ifcToto: 9, midSoil: 8, midToto: 8, scalarv: 1)
Dimensions without coordinates: hru, ifcToto, midSoil, midToto, scalarv
Data variables:
    mLayerVolFracIce      (midToto, hru) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
    scalarCanairTemp      (scalarv, hru) float64 286.0
    nSnow                 (scalarv, hru) int64 0
    iLayerHeight          (ifcToto, hru) float64 0.0 0.025 0.1 ... 1.5 2.5 4.0
    mLayerMatricHead      (midSoil, hru) float64 -100.0 -100.0 ... -100.0 -100.0
    scalarSnowAlbedo      (scalarv, hru) float64 0.82
    dt_init               (scalarv, hru) float64 360.0
    mLayerTemp            (midToto, hru) float64 288.5 288.5 ... 288.5 288.5
    scalarSfcMeltPond     (scalarv, hru) float64 0.0
    scalarCanopyTemp      (scalarv, hru) float64 290.0
    scalarSnowDepth       (scalarv, hru) float64 0.0
    nSoil                 (scalarv, hru) int64 8
    scalarSWE             (scalarv, hru) float64 0.0
    scalarCanopyLiq       (scalarv, hru) float64 3.16
    mLayerVolFracLiq      (midToto, hru) float64 0.29 0.29 0.29 ... 0.29 0.29
    mLayerDepth           (midToto, hru) float64 0.025 0.075 0.15 ... 1.0 1.5
    scalarCanopyIce       (scalarv, hru) float64 0.0
    scalarAquiferStorage  (scalarv, hru) float64 0.0

### 3.2 Show iLayerHeight (Height of the layer interface) Variable

In [4]:
initial_conditions['iLayerHeight']

<xarray.DataArray 'iLayerHeight' (ifcToto: 9, hru: 1)>
array([[0.   ],
       [0.025],
       [0.1  ],
       [0.25 ],
       [0.5  ],
       [1.   ],
       [1.5  ],
       [2.5  ],
       [4.   ]])
Dimensions without coordinates: ifcToto, hru

### 3.3 Create restart file

In [5]:
s.run("local", run_suffix="restart_test", iHRU='1', freq_restart="e")

### 3.4  Show Restart file

In [6]:
restart_files_nc = s.get_output_files()
restart_files_nc

['/home/jovyan/work/SUMMA3_Manual/SummaModel_ReynoldsAspenStand_StomatalResistance/output/vegImpactsTranspire_restart_test_H1_timestep.nc']

In [7]:
import xarray as xr
restart_file1 = xr.open_dataset(restart_files_nc[0])
restart_file1

<xarray.Dataset>
Dimensions:                    (gru: 1, hru: 1, ifcSoil: 9, ifcToto: 14, midSoil: 8, midToto: 13, time: 10945)
Coordinates:
  * time                       (time) datetime64[ns] 2006-07-01 ... 2007-09-30
  * hru                        (hru) int32 1
  * gru                        (gru) int32 0
Dimensions without coordinates: ifcSoil, ifcToto, midSoil, midToto
Data variables:
    pptrate                    (time, hru) float64 ...
    airtemp                    (time, hru) float64 ...
    basin__SurfaceRunoff       (time, gru) float64 ...
    basin__ColumnOutflow       (time, gru) float64 ...
    basin__AquiferStorage      (time, gru) float64 ...
    basin__AquiferRecharge     (time, gru) float64 ...
    basin__AquiferBaseflow     (time, gru) float64 ...
    basin__AquiferTranspire    (time, gru) float64 ...
    averageInstantRunoff       (time, gru) float64 ...
    averageRoutedRunoff        (time, gru) float64 ...
    nSnow                      (time, hru) int32 ...
    nSoil                      (time, hru) int32 ...
    scalarSWE                  (time, hru) float64 ...
    mLayerTemp                 (time, midToto, hru) float64 ...
    mLayerVolFracIce           (time, midToto, hru) float64 ...
    mLayerVolFracLiq           (time, midToto, hru) float64 ...
    mLayerVolFracWat           (time, midToto, hru) float64 ...
    mLayerMatricHead           (time, midSoil, hru) float64 ...
    scalarSurfaceTemp          (time, hru) float64 ...
    mLayerDepth                (time, midToto, hru) float64 ...
    iLayerHeight               (time, ifcToto, hru) float64 ...
    scalarSenHeatTotal         (time, hru) float64 ...
    scalarLatHeatTotal         (time, hru) float64 ...
    scalarSnowSublimation      (time, hru) float64 ...
    scalarCanopyTranspiration  (time, hru) float64 ...
    scalarCanopyEvaporation    (time, hru) float64 ...
    scalarGroundEvaporation    (time, hru) float64 ...
    scalarThroughfallSnow      (time, hru) float64 ...
    scalarThroughfallRain      (time, hru) float64 ...
    scalarRainPlusMelt         (time, hru) float64 ...
    scalarInfiltration         (time, hru) float64 ...
    scalarExfiltration         (time, hru) float64 ...
    scalarSurfaceRunoff        (time, hru) float64 ...
    iLayerLiqFluxSoil          (time, ifcSoil, hru) float64 ...
    mLayerLiqFluxSoil          (time, midSoil, hru) float64 ...
    hruId                      (hru) int64 ...
    gruId                      (gru) int64 ...
Attributes:
    summaVersion:     v3.0.0
    buildTime:        Wed Jul 29 15:28:12 UTC 2020
    gitBranch:        master-0-ga843a8c
    gitHash:          a843a8c46484d47979f8cb8e48b82ef71a177894
    soilStress:       NoahType
    stomResist:       Jarvis
    num_method:       itertive
    fDerivMeth:       analytic
    LAI_method:       specified
    f_Richards:       mixdform
    groundwatr:       noXplict
    hc_profile:       constant
    bcUpprTdyn:       nrg_flux
    bcLowrTdyn:       zeroFlux
    bcUpprSoiH:       liq_flux
    bcLowrSoiH:       drainage
    veg_traits:       CM_QJRMS1988
    canopyEmis:       difTrans
    snowIncept:       lightSnow
    windPrfile:       logBelowCanopy
    astability:       louisinv
    canopySrad:       BeersLaw
    alb_method:       varDecay
    snowLayers:       CLM_2010
    compaction:       anderson
    thCondSnow:       jrdn1991
    thCondSoil:       mixConstit
    spatial_gw:       localColumn
    subRouting:       timeDlay
    notPopulatedYet:  notPopulatedYet

In [8]:
import os
output_path = os.path.join(s.manager['outputPath'].value, 'vegImpactsTranspire_summaRestart_2006-09-30-00_restart_test_H1.nc')
output_path

'/home/jovyan/work/SUMMA3_Manual/SummaModel_ReynoldsAspenStand_StomatalResistance/output/vegImpactsTranspire_summaRestart_2006-09-30-00_restart_test_H1.nc'

In [9]:
restart_file2 = xr.open_dataset(output_path)
restart_file2

<xarray.Dataset>
Dimensions:                    (hru: 1, ifcToto: 14, midSoil: 8, midToto: 13, scalarv: 1, spectral: 2)
Dimensions without coordinates: hru, ifcToto, midSoil, midToto, scalarv, spectral
Data variables:
    dt_init                    (scalarv, hru) float64 ...
    scalarCanopyIce            (scalarv, hru) float64 ...
    scalarCanopyLiq            (scalarv, hru) float64 ...
    scalarCanopyWat            (scalarv, hru) float64 ...
    scalarCanairTemp           (scalarv, hru) float64 ...
    scalarCanopyTemp           (scalarv, hru) float64 ...
    spectralSnowAlbedoDiffuse  (spectral, hru) float64 ...
    scalarSnowAlbedo           (scalarv, hru) float64 ...
    scalarSnowDepth            (scalarv, hru) float64 ...
    scalarSWE                  (scalarv, hru) float64 ...
    scalarSfcMeltPond          (scalarv, hru) float64 ...
    mLayerTemp                 (midToto, hru) float64 ...
    mLayerVolFracIce           (midToto, hru) float64 ...
    mLayerVolFracLiq           (midToto, hru) float64 ...
    mLayerVolFracWat           (midToto, hru) float64 ...
    mLayerMatricHead           (midSoil, hru) float64 ...
    scalarAquiferStorage       (scalarv, hru) float64 ...
    scalarSurfaceTemp          (scalarv, hru) float64 ...
    mLayerDepth                (midToto, hru) float64 ...
    mLayerHeight               (midToto, hru) float64 ...
    iLayerHeight               (ifcToto, hru) float64 ...
    nSnow                      (hru) int32 ...
    nSoil                      (hru) int32 ...